In [ ]:
import pandas as pd 
import plotly.express as px


In [ ]:
shared_drive = r"G:\.shortcut-targets-by-id\184tVjsIO-GAjbkSakwDbEZ40M5mPpgu4\Capstone\cleaned_data"
google_drive = r"G:\My Drive\Spring_2022\CS554\Project\data"

drive = r"D:\Users\yiboz\Programming\Github\CS554\data"

In [ ]:
articles = pd.read_csv(shared_drive+r"\articles_clean.csv")
customers = pd.read_csv(shared_drive+r"\customers_clean.csv")
transactions = pd.read_csv(shared_drive+r"\transactions_train.csv")

In [ ]:
"""
Num to num bivariate analysis 
(Customers.age and Transactions.price) 

https://www.pythonfordatascience.org/variance-covariance-correlation/#correlation

Observations: 
Covariance: 0.013649
Correlation: 0.054944 

Low absolute values of covariance and correlation suggests a weak linear relationship between these values


"""

In [ ]:
trans_join_cust_cats = transactions.join(customers[['customer_id', 'age', 'club_member_status', 'fashion_news_frequency']].set_index('customer_id'), on='customer_id')
trans_join_cust_cats.head()

In [ ]:
full_trans_join = trans_join_cust_cats.join(articles[['article_id', 'index_group_name', 'colour_group_name']].set_index('article_id'), on='article_id')
full_trans_join.head()


In [ ]:
def num_to_num(df, nums, plot=False, frac=0.01): 
    df = df[nums]
    var = df.var() 
    cov = df.cov()
    corr = df.corr()
    
    if plot: 
        sampleDf = df.sample(frac=frac)
        fig = px.scatter_matrix(sampleDf)
        fig.show()
    d = {'nums':nums, 'Variance': var.values.tolist(), 'Covariance': cov[nums[0]].tolist(), 'Correlation': corr[nums[0]].tolist()}
    return pd.DataFrame(data=d)
    

In [ ]:
num_to_num(full_trans_join, ['price', 'age'], plot=True)

In [ ]:
transactions_nums.describe()

In [ ]:
sampleTransactions = transactions_nums.sample(frac=0.01)
sampleTransactions.describe()

In [ ]:
"""
Categorical and Numerical Data 

Violin graphs to visualize distribution 
full_trans_join = transactions left join customers.age on customer_id and articles on article_id 

Perhaps need to sample data 

|Cat|Num|
|---|---|
|t_dat|price|
|sales_channel_id|price|
|club_member_status|price|
|fashion_news_frequency|price|
|index_group_name|price|
|colour_group_name|price|
|all the above|age|

"""

In [ ]:
cats = ["sales_channel_id", "club_member_status", "fashion_news_frequency", "index_group_name", "colour_group_name"]
nums = ["price", "age"]

In [ ]:
def no_hovering(df): 
    no_hover = {}
    for cols in df.columns: 
        no_hover[cols] = False
    return no_hover

In [ ]:
def keep_top_cats(df, col, num): 
    # keep the top num largest categories 
    # rename the rest into 'Other'
    keep = df[col].value_counts().head(num).index.tolist()
    df1 = df
    cond = ~df1[col].isin(keep)
    df1.loc[cond, col] = 'Other' 
    
    return df1
    

In [ ]:
def cat_num_distributions(df, cats, nums, max_cols=10, hovering=False): 
    figs = []
    no_hover = {}
    if ~hovering: 
        no_hover = no_hovering(df)
    
    for n in nums: 
        for c in cats: 
            
            fig_title = str(n).upper() + " X " + str(c).upper()
            if df[c].value_counts().size > max_cols: 
                # only get top default 10 largest values 
                # aggregate rest into 'Other'
                tempDf = keep_top_cats(df, c, max_cols)
                fig = px.violin(tempDf, y=n, x=c, box=True, points=False, title=fig_title, hover_data = no_hover)
                figs.append(fig)
            else:            
                fig = px.violin(df, y=n, x=c, box=True, points=False, title=fig_title, hover_data = no_hover)
                figs.append(fig)
    return figs 

In [ ]:
full_trans_join.describe()

In [ ]:
sampleTrans = full_trans_join.sample(frac=0.01)
sampleTrans.describe()

In [ ]:
all_figures = cat_num_distributions(sampleTrans, cats, nums)

In [ ]:
# for f in all_figures: 
#    f.show()